In [ ]:
%pip install git+https://github.com/vladkens/twscrape.git
%pip install langdetect twscrape parsel tqdm pandas
!git clone https://github.com/AI4Finance-Foundation/FinNLP.git

Functions to clean the contents of the tweet

In [2]:
import re
import html
def clean_tweet_content(text):
    """Remove mentions, links, emojis, newlines, and specific search terms from tweet text."""
    text = html.unescape(text)
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'https?://\S+', '', text)  # Remove URLs
    text = re.sub(r'\n+', ' ', text)  # Replace newlines with space
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'\$\w+', '', text)  # Remove cashtags

    # Remove emojis
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"
                                u"\U0001F300-\U0001F5FF"
                                u"\U0001F680-\U0001F6FF"
                                u"\U0001F1E0-\U0001F1FF"
                                u"\U0001F700-\U0001F77F"
                                u"\U0001F780-\U0001F7FF"
                                u"\U0001F800-\U0001F8FF"
                                u"\U0001F900-\U0001F9FF"
                                u"\U0001FA00-\U0001FA6F"
                                u"\U0001FA70-\U0001FAFF"
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)  # Apply regex to remove emojis
    
    # Normalize spaces - replace two or more spaces with a single space
    text = re.sub(r'\s{2,}', ' ', text)

    return text.strip()

def is_spam(text):
    # Define a list of promotional phrases to search for
    promotional_phrases = [
        "join us",
        "find out more",
        "Sign up now",
        "become a member",
        "Subscribe for updates",
        "Limited time offer",
        "Exclusive access",
        "Free trial",
        "Don't miss out",
        "Last chance",
        "Early bird discount",
        "Save big",
        "Register today",
        "Unlock benefits",
        "Special promotion",
        "Available for a limited time",
        "Claim your offer",
        "Act now",
        "Exclusive deals",
        "Get started",
        "Offer ends soon",
        "While supplies last",
        "Money-back guarantee",
        "Check out now",
        "Check out now...",
        "Check out now..",
        "Check out now.",
        "Check out",
        "Pay a visit",
        "Most profitable trading community.",
        "Join the",
        "Best trading community",
        "Free access",
        "trading community",
        "our ChatRoom",
        "join here",
        "Join our",
        "Check us out",
        "Daily ALERTS",
        "trading alerts",
        "Free chatroom",
        "Learn To Trade",
        "limited time",
        "CHATROOM",
        "We provide",
        "with us",
        "OUR FREE",
        "Learn how",
        'chat room',
        "Top analyst",
        "price target"
    ]
    
    promotional_words = [
        'trading',
        'community',
        'alerts'
    ]
    
    # Create a combined regular expression from the list of phrases
    regex_pattern = '|'.join(map(re.escape, promotional_phrases))
    # Search for any of the phrases in the text
    contains_promotion = re.search(regex_pattern, text, re.IGNORECASE) is not None
    
        # Create a combined regular expression from the list of phrases with optional spaces between words
    regex_pattern = '|'.join(['\\s*'.join(map(re.escape, phrase.split())) for phrase in promotional_words])
    # Find all occurrences of the phrases in the text and check if there are two or more
    contains_two_or_more_promotion_words = len(re.findall(regex_pattern, text, re.IGNORECASE)) >= 2
    
    # Regular expression to find cashtags (e.g., $AAPL)
    cashtag_pattern = r'\$[A-Za-z]+'
    # Find all occurrences of cashtags in the text
    cashtags = re.findall(cashtag_pattern, text)
    # Check if there are more than four cashtags
    contains_more_than_four_cashtags = len(cashtags) > 4

    # _________________________________________________________________________________
    
    # A simple heuristic: check if the text has at least three alphabetic characters
    is_too_short = len(re.findall("[a-zA-Z]", text)) < 5
    
    return  contains_promotion or contains_two_or_more_promotion_words or contains_more_than_four_cashtags or is_too_short

Get Tweets from Twitter

In [3]:
st_search_term = "Tesla OR TSLA"
ticker="TSLA"
tweets_limit=30

In [7]:
from twscrape import API, gather
from twscrape.logger import set_log_level
import json
import time
from langdetect import detect, LangDetectException
import re
import pytz
import datetime


# ####################################################################################
# Gather the tweets
# ####################################################################################
async def main():
    api = API()  # or API("path-to.db") - default is `accounts.db`

    # ADD ACCOUNTS (for CLI usage see BELOW)
    await api.pool.add_account("DarOho4050", "asdfghjkl1!", "we444465@gmail.com", "mail_pass1")
    await api.pool.login_all()

# search (latest tab)
    tweets = await gather(api.search(st_search_term, limit=tweets_limit))  # list[Tweet]
    # print(tweets)
    
    def user_to_dict(user):
        return {
            'id': user.id,
            'id_str': user.id_str,
            'url': user.url,
            'username': user.username,
            'displayname': user.displayname,
            'rawDescription': user.rawDescription,
            'created': user.created.isoformat() if user.created else None,            
        }
    
    def tweet_to_dict(tweet):
        return {
            'id': tweet.id,
            'date': tweet.date.isoformat(),
            'rawContent': tweet.rawContent,
            'cleanContent': clean_tweet_content(tweet.rawContent),
            'url': tweet.url,
            'user': user_to_dict(tweet.user),
            'lang': tweet.lang,
            'replyCount': tweet.replyCount,
            'retweetCount': tweet.retweetCount,
            'likeCount': tweet.likeCount,
            'quoteCount': tweet.quoteCount
        }

    # Convert each Tweet instance to a dictionary
    tweets_as_dicts = [tweet_to_dict(tweet) for tweet in tweets]
    # print(tweets_as_dicts)
    return tweets_as_dicts
    
def write_tweets_to_file(tweets, file_path):
    """Generic function to write tweets to a specified file with deduplication."""
    new_tweets_count = 0
    existing_ids = set()
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_ids.update(json.loads(line)['id'] for line in file)
    except FileNotFoundError:
        pass  # It's okay if the file does not exist yet

    with open(file_path, 'a', encoding='utf-8') as file:
        for tweet in tweets:
            if tweet['id'] not in existing_ids:
                file.write(json.dumps(tweet, ensure_ascii=False) + '\n')
                new_tweets_count += 1
                existing_ids.add(tweet['id'])

    return new_tweets_count

def does_tweet_match_criteria(tweet_content, search_terms):
    """Check if tweet content matches the search criteria using regex for flexibility.
       Supports 'AND' and 'OR' logic in search terms."""
    # Split the search terms based on 'AND' and 'OR' logic
    or_groups = [group.strip() for group in search_terms.split('OR')]
    
    for group in or_groups:
        # For each 'OR' group, check if any of the 'AND' terms are all present
        and_terms = group.split('AND')
        if all(re.search(r'\b' + term.strip() + r'\b', tweet_content, re.IGNORECASE) for term in and_terms):
            return True  # If all 'AND' terms in any 'OR' group match, return True
    return False  # If none of the groups match, return False

def filter_tweets(tweets, search_term, language="en"):
    """Filter out Tweets that do not contain all of the provided search terms."""
    english_tweets = []
    filtered_tweets = []  # Tweets filtered out
    for tweet in tweets:
        if does_tweet_match_criteria(tweet['rawContent'], search_term):
            try:
                if detect(tweet['cleanContent']) == 'en' and not is_spam(tweet['rawContent']):
                    english_tweets.append(tweet)
                else:
                    filtered_tweets.append(tweet)
            except LangDetectException:
                # Assume English if detection fails
                filtered_tweets.append(tweet)
        else:
            filtered_tweets.append(tweet)
    return english_tweets, filtered_tweets

async def main_tweets():
    tweets = await main()  # Assume this fetches a list of tweet dictionaries
    english_tweets, filtered_tweets = filter_tweets(tweets, st_search_term)
    
    new_english_count = write_tweets_to_file(english_tweets, f'{ticker}_tweets.jsonl')
    filtered_count = write_tweets_to_file(filtered_tweets, f'trash_{ticker}_tweets.jsonl')
    
    total_processed = len(english_tweets) + len(filtered_tweets)
    
    if total_processed > 0:
        english_percentage = (new_english_count / total_processed) * 100
        filtered_percentage = (filtered_count / total_processed) * 100
        print(f"Processed {total_processed} tweets. New Tweets/New Percentage: {new_english_count + filtered_count}:{(english_percentage + filtered_percentage):.2f}% New English: {english_percentage:.2f}%. Filtered: {filtered_percentage:.2f}%.")
    else: 
        print("No tweets to process.")
    
    return {"new_tweets": new_english_count + filtered_count, "total_tweets": total_processed}

Get Tweets from StockTwits

In [2]:
# Stocktwits
import sys
import os

sys.path.append(f'{os.getcwd()}\FinNLP')
from finnlp.data_sources.social_media.stocktwits_streaming import Stocktwits_Streaming

st_search_term = f"{ticker}"
file_path = f'./{ticker}_stocktwits.jsonl'


pages = 1
config = {
    "max_retry": 5
}

async def main_stocktwits():
    downloader = Stocktwits_Streaming(config)
    downloader.download_streaming_stock(ticker, pages)

    stocktweets = downloader.dataframe
    stocktweets = stocktweets.rename(columns={'body': 'rawContent'})
    stocktweets = stocktweets.to_dict('records')
    # print(stocktweets)

    stocktweets = [{**tweet, 'cleanContent': clean_tweet_content(tweet['rawContent'])} for tweet in stocktweets]

    english_tweets, filtered_tweets = filter_tweets(stocktweets, st_search_term)

    new_english_count = write_tweets_to_file(english_tweets, f'{ticker}_stocktweets.jsonl')
    filtered_count = write_tweets_to_file(filtered_tweets, f'trash_{ticker}_stocktweets.jsonl')

    total_processed = len(english_tweets) + len(filtered_tweets)

    if total_processed > 0:
        english_percentage = (new_english_count / total_processed) * 100
        filtered_percentage = (filtered_count / total_processed) * 100
        print(f"Processed {total_processed} stocktwits. New Tweets/New Percentage: {new_english_count + filtered_count}:{(english_percentage + filtered_percentage):.2f}% New English: {english_percentage:.2f}%. Filtered: {filtered_percentage:.2f}%.")
    else: 
        print("No tweets to process.")
        
    return {"new_tweets": new_english_count + filtered_count, "total_tweets": total_processed}

NameError: name 'ticker' is not defined

Run both at the same time (indefinity every 60 seconds)

In [12]:
from datetime import datetime

async def save_to_jsonl(data, file_path='tweets_stats.jsonl'):
    with open(file_path, 'a', encoding='utf-8') as file:
        file.write(json.dumps(data) + '\n')

while True:
    tweet_data = await main_tweets()
    stocktwit_data = await main_stocktwits()
    
    combined_data = {
        "twitter": tweet_data,
        "stocktwits": stocktwit_data,
        "timestamp": datetime.now(pytz.utc).strftime("%Y-%m-%d %H:%M:%S %Z")
    }
    
    await save_to_jsonl(combined_data)
    
    time.sleep(10)
    time.sleep(10)
    time.sleep(10)  
    time.sleep(10)
    time.sleep(10)
    time.sleep(10)

2024-02-18 23:12:06.889 | WARNING  | twscrape.accounts_pool:add_account:76 - Account DarOho4050 already exists
2024-02-18 23:12:08.340 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:275 - No account available for queue "SearchTimeline". Next available at 23:13:26
2024-02-18 23:13:28.532 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:281 - Continuing with account DarOho4050 on queue SearchTimeline


[Tweet(id=1759355106225447278, id_str='1759355106225447278', url='https://twitter.com/tomstoppard4/status/1759355106225447278', date=datetime.datetime(2024, 2, 18, 23, 11, 59, tzinfo=datetime.timezone.utc), user=User(id=1629202673642917888, id_str='1629202673642917888', url='https://twitter.com/tomstoppard4', username='tomstoppard4', displayname='tomstoppard', rawDescription='', created=datetime.datetime(2023, 2, 24, 19, 32, 48, tzinfo=datetime.timezone.utc), followersCount=8, friendsCount=0, statusesCount=13762, favouritesCount=1, listedCount=0, mediaCount=13741, location='', profileImageUrl='https://pbs.twimg.com/profile_images/1629202783072288768/fQR54Axc_normal.jpg', profileBannerUrl=None, protected=None, verified=False, blue=False, blueType=None, descriptionLinks=[], _type='snscrape.modules.twitter.User'), lang='en', rawContent='Join our Alerts &amp; Options\n\nhttps://t.co/NsHPaqZX2n\n\n$SPY $NFLX $M $DIS $OPGN $IBIO $EPAZ $NIO $AMD $MSFT $TSLA https://t.co/dYwyrgts0K', replyCoun

100%|██████████| 1/1 [00:00<00:00,  1.67it/s]


Processed 30 stocktwits. New Tweets/New Percentage: 0:0.00% New English: 0.00%. Filtered: 0.00%.


2024-02-18 23:14:31.222 | WARNING  | twscrape.accounts_pool:add_account:76 - Account DarOho4050 already exists


...

2024-02-18 23:23:06.630 | WARNING  | twscrape.accounts_pool:add_account:76 - Account DarOho4050 already exists


[Tweet(id=1759357817968845217, id_str='1759357817968845217', url='https://twitter.com/tomstoppard4/status/1759357817968845217', date=datetime.datetime(2024, 2, 18, 23, 22, 45, tzinfo=datetime.timezone.utc), user=User(id=1629202673642917888, id_str='1629202673642917888', url='https://twitter.com/tomstoppard4', username='tomstoppard4', displayname='tomstoppard', rawDescription='', created=datetime.datetime(2023, 2, 24, 19, 32, 48, tzinfo=datetime.timezone.utc), followersCount=8, friendsCount=0, statusesCount=13784, favouritesCount=1, listedCount=0, mediaCount=13763, location='', profileImageUrl='https://pbs.twimg.com/profile_images/1629202783072288768/fQR54Axc_normal.jpg', profileBannerUrl=None, protected=None, verified=False, blue=False, blueType=None, descriptionLinks=[], _type='snscrape.modules.twitter.User'), lang='en', rawContent='Most winning trading chat!!! For a limited time, we are opening our trading chatroom to the public! \nhttps://t.co/NsHPaqZX2n\n\n$TSLA $MSFT $AMZN $GOOG $

100%|██████████| 1/1 [00:00<00:00,  1.26it/s]


Processed 30 stocktwits. New Tweets/New Percentage: 1:3.33% New English: 3.33%. Filtered: 0.00%.


2024-02-18 23:24:09.855 | WARNING  | twscrape.accounts_pool:add_account:76 - Account DarOho4050 already exists
2024-02-18 23:25:49.070 | WARNING  | twscrape.queue_client:req:224 - Unhandled error <class 'httpx.ConnectTimeout'>: 
2024-02-18 23:25:49.089 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:275 - No account available for queue "SearchTimeline". Next available at 23:40:49
